In [1]:
from helpers import *

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from CustomLogisticRegression import CustomLogisticRegression as CLR
import sklearn.discriminant_analysis as DA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

# Compute discrimination

In [2]:
def computeDiscrimination(X_test, prediction, sensitiveAttr):
    X_test = X_test.assign(prediction = prediction)
    protectedGroup = X_test[X_test[sensitiveAttr] == 1]
    unprotectedGroup = X_test[X_test[sensitiveAttr] == 0]
    proportionOfProtected = protectedGroup['prediction'].sum() / protectedGroup[sensitiveAttr].count()
    proportionOfUnprotected = unprotectedGroup['prediction'].sum() / protectedGroup[sensitiveAttr].count()
    discrim = abs(proportionOfProtected - proportionOfUnprotected)
    return discrim
    

# Run Helper

In [3]:
def run(X, y, X_test, y_test, SA):
    kfold = model_selection.KFold(n_splits=5)
#     kfold = model_selection.StratifiedKFold(n_splits=5)
    for i, model in enumerate(models):
        cv_result = model_selection.cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

        model.fit(X, y)
        prediction_test = model.predict(X_test)

        acc_score_val = np.mean(cv_result)
        acc_score_test = accuracy_score(y_test, prediction_test)
        discr_score = computeDiscrimination(X_test, prediction_test, SA)
        print ('-'*40)
        print ('val: {0}: {1}'.format(names[i], acc_score_val))
        print ('test: {0}: {1}'.format(names[i], acc_score_test))
        print('{0} discrimination: {1}'.format(names[i], discr_score))
        

# Runner with the data splitting according to the report

In [ ]:
# def runAdultWithSplitting(X, y, X_test, y_test, SA):
#     X_splits = np.array_split(X, 5)
#     y_splits = np.array_split(y, 5)
#     for x_split, y_split in zip(X_splits, y_splits):
#         val_amount = math.floor((1/3) * x_split.shape[0]) # 1/3 of 1 split set
#         X_val = x_split[0:val_amount]
#         y_val = y_split[0:val_amount]
#         X_train = x_split[val_amount:]
#         y_train = y_split[val_amount:]
# #         splitSize = x_split.shape[0]
# #         X_train = x_split[0:splitSize-val_amount]
# #         y_train = y_split[0:splitSize-val_amount]
# #         X_val = x_split[splitSize-val_amount:]
# #         y_val = y_split[splitSize-val_amount:]
#         for i, model in enumerate(models):
#             model.fit(X_train, y_train)
#             prediction_val = model.predict(X_val)
#             acc_score_val = np.mean(y_val == prediction_val)
#             prediction_test = model.predict(X_test)
#             acc_score_test = accuracy_score(y_test, prediction_test)
#             discr_score = computeDiscrimination(X_test, prediction_test, SA)
#             print ('-'*40)
#             print ('val: {0}: {1}'.format(names[i], acc_score_val))
#             print ('test: {0}: {1}'.format(names[i], acc_score_test))
#             print('{0} discrimination: {1}'.format(names[i], discr_score))
#         print ('-'*80)
#         print ('-'*80)

In [4]:
def runAdultWithSplitting(X, y, X_test, y_test, SA):
    X_splits = np.array_split(X, 5)
    y_splits = np.array_split(y, 5)
    for i, model in enumerate(models):
        acc_list = []
        disc_list = []
        for x_split, y_split in zip(X_splits, y_splits):
            val_amount = math.floor((1/3) * x_split.shape[0]) # 1/3 of 1 split set
            X_val = x_split[0:val_amount]
            y_val = y_split[0:val_amount]
            X_train = x_split[val_amount:]
            y_train = y_split[val_amount:]
    #         splitSize = x_split.shape[0]
    #         X_train = x_split[0:splitSize-val_amount]
    #         y_train = y_split[0:splitSize-val_amount]
    #         X_val = x_split[splitSize-val_amount:]
    #         y_val = y_split[splitSize-val_amount:]
            model.fit(X_train, y_train)
            prediction_val = model.predict(X_val)
            acc_score_val = np.mean(y_val == prediction_val)
            acc_list.append(acc_score_val)
#         prediction_test = model.predict(X_test)
#         acc_score_test = accuracy_score(y_test, prediction_test)
            discr_score = computeDiscrimination(X_val, prediction_val, SA)
            disc_list.append(discr_score)
        
        model.fit(X, y)
        prediction_test = model.predict(X_test)
        acc_score_test = np.mean(y_test == prediction_test)
        print ('-'*40)
        print ('val: {0}: {1}'.format(names[i], np.mean(acc_list)))
        print ('test: {0}: {1}'.format(names[i], acc_score_test))
        print('{0} discrimination: {1}'.format(names[i], np.mean(disc_list)))
        print ('-'*80)
        print ('-'*80)

In [ ]:
def runGermanWithSplitting(X, y, SA):
    X_splits = np.array_split(X, 5)
    y_splits = np.array_split(y, 5)
    for x_split, y_split in zip(X_splits, y_splits):
        train_amount = math.floor(0.5 * x_split.shape[0]) # 50% of 1 split set (round)
        X_train = x_split[0:train_amount]
        y_train = y_split[0:train_amount]
        val_amount = math.floor(0.2 * x_split.shape[0]) # 20% of 1 split set
        X_val = x_split[train_amount:train_amount+val_amount]
        y_val = y_split[train_amount:train_amount+val_amount]
        X_test = x_split[train_amount+val_amount:] # 30% of 1 split set
        y_test = y_split[train_amount+val_amount:]
        for i, model in enumerate(models):
            model.fit(X_train, y_train)
            prediction_val = model.predict(X_val)
            acc_score_val = np.mean(y_val == prediction_val)
            prediction_test = model.predict(X_test)
#             acc_score_test = accuracy_score(y_test, prediction_test)
            acc_score_test = 1 - (np.mean(abs(y_test - prediction_test)))
            discr_score = computeDiscrimination(X_test, prediction_test, SA)
            print ('-'*40)
            print ('val: {0}: {1}'.format(names[i], acc_score_val))
            print ('test: {0}: {1}'.format(names[i], acc_score_test))
            print('{0} discrimination: {1}'.format(names[i], discr_score))
        print ('-'*80)
        print ('-'*80)

# Models to Run

In [5]:
models = []
names = ['LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10)',
         'LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10)',
         'LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10)',
         'LR(solver=newton-cg, fit_intercept=False, max_iter=100, penalty=none, C=1e10)',
         'LR(solver=newton-cg, fit_intercept=False, max_iter=500, penalty=none, C=1e10)',
         'LR(solver=newton-cg, fit_intercept=False, max_iter=1500, penalty=none, C=1e10)',
         'LR(solver=liblinear, fit_intercept=False, max_iter=1000, C=1e10)',
         'LR(solver=liblinear, fit_intercept=False, max_iter=2000, C=1e10)',
         'LR(solver=liblinear, fit_intercept=False, max_iter=3000, C=1e10)',
         'LR(solver=sag, fit_intercept=False, max_iter=1000, penalty=none, C=1e10)',
         'LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10)',
         'LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10)',
         'LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10)',
         'LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10)',
         'LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10)',
         'Custom Logistic Regression(lr=0.1, max_itr=100)',
         'Custom Logistic Regression(lr=0.1, max_itr=500)',
         'Custom Logistic Regression(lr=0.1, max_itr=1500)',
         'Custom Logistic Regression(lr=0.01, max_itr=100)',
         'Custom Logistic Regression(lr=0.01, max_itr=500)',
         'Custom Logistic Regression(lr=0.01, max_itr=1500)',
         'Custom Logistic Regression(lr=0.001, max_itr=100)',
         'Custom Logistic Regression(lr=0.001, max_itr=500)',
         'Custom Logistic Regression(lr=0.001, max_itr=1500)',
         'Custom Logistic Regression(lr=0.0001, max_itr=100)',
         'Custom Logistic Regression(lr=0.0001, max_itr=500)',
         'Custom Logistic Regression(lr=0.0001, max_itr=1500)',
#          'Linear Discriminant Analysis',
#          'Quadratic Discriminant Analysis',
#          'Random Forest',
#          'Neural Network',
#          'Gaussian NB',
#          'Bernoulli NB',
#          'Complement NB',
#          'Multinomial NB',
#          'Random Forest',
#          'K Neighbors Classifier',
#          'SVM',
#          'LinearSVC'
]

models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=700, penalty='none'))
models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=1400, penalty='none'))
models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=2100, penalty='none'))
models.append(LogisticRegression(solver='newton-cg', fit_intercept=False, max_iter=100, penalty='none'))
models.append(LogisticRegression(solver='newton-cg', fit_intercept=False, max_iter=500, penalty='none'))
models.append(LogisticRegression(solver='newton-cg', fit_intercept=False, max_iter=1500, penalty='none'))
models.append(LogisticRegression(solver='liblinear', fit_intercept=False, max_iter=1000, C=1e10))
models.append(LogisticRegression(solver='liblinear', fit_intercept=False, max_iter=2000, C=1e10))
models.append(LogisticRegression(solver='liblinear', fit_intercept=False, max_iter=3000, C=1e10))
models.append(LogisticRegression(solver='sag', fit_intercept=False, max_iter=1000, penalty='none'))
models.append(LogisticRegression(solver='sag', fit_intercept=False, max_iter=3000, penalty='none'))
models.append(LogisticRegression(solver='sag', fit_intercept=False, max_iter=5000, penalty='none'))
models.append(LogisticRegression(solver='saga', fit_intercept=False, max_iter=700, penalty='none'))
models.append(LogisticRegression(solver='saga', fit_intercept=False, max_iter=1400, penalty='none'))
models.append(LogisticRegression(solver='saga', fit_intercept=False, max_iter=2100, penalty='none'))
models.append(CLR(0.1, 100))
models.append(CLR(0.1, 500))
models.append(CLR(0.1, 1500))
models.append(CLR(0.01, 100))
models.append(CLR(0.01, 500))
models.append(CLR(0.01, 1500))
models.append(CLR(0.001, 100))
models.append(CLR(0.001, 500))
models.append(CLR(0.001, 1500))
models.append(CLR(0.0001, 100))
models.append(CLR(0.0001, 500))
models.append(CLR(0.0001, 1500))
# models.append(DA.LinearDiscriminantAnalysis())
# models.append(DA.QuadraticDiscriminantAnalysis())
# models.append(RandomForestClassifier(n_estimators=100))
# models.append(MLPClassifier())
# models.append(GaussianNB())
# models.append(BernoulliNB())
# models.append(ComplementNB())
# models.append(MultinomialNB())
# models.append(DecisionTreeClassifier())
# models.append(KNeighborsClassifier(n_neighbors=50))
# models.append(SVC())
# models.append(LinearSVC())

# Adult Dataset

In [6]:
df_adult, pct = load_adult('datasets/adult/adult.data')
X_adult = df_adult.iloc[:, :-1]
y_adult = df_adult.iloc[:, -1]
print('percentage of corrupt rows in training: {0:.1f}%'.format((1-pct)*100))

df_adult_test, pct = load_adult('datasets/adult/adult.test')
X_adult_test = df_adult_test.iloc[:, :-1]
y_adult_test = df_adult_test.iloc[:, -1]
print('percentage of corrupt rows in testing: {0:.1f}%'.format((1-pct)*100))

percentage of corrupt rows in training: 7.4%
percentage of corrupt rows in testing: 7.5%


# expanding

In [7]:
X_adult, X_adult_test = encode_adult(X_adult, X_adult_test)

X_adult_all = X_adult.append(X_adult_test)
X_adult_all_expand = pd.get_dummies(X_adult_all)

# X_expand, X_expand_test = encode_adult(X_expand, X_expand_test)
X_adult_expand = X_adult_all_expand[0:X_adult.shape[0]]
X_adult_expand_test = X_adult_all_expand[X_adult.shape[0]:]

# LabelEncoder

In [ ]:
# encoders = {"workclass": preprocessing.LabelEncoder(), 
#             "education": preprocessing.LabelEncoder(), 
#             "marital-status": preprocessing.LabelEncoder(), 
#             "occupation": preprocessing.LabelEncoder(), 
#             "relationship": preprocessing.LabelEncoder(), 
#             "race": preprocessing.LabelEncoder(), 
#             "sex": preprocessing.LabelEncoder(), 
#             "native-country": preprocessing.LabelEncoder()}

# X_encoded = encode(X_adult, encoders)
# X_encoded_test = encode(X_adult_test, encoders)

In [8]:
# run(X_expand, y_adult, X_expand_test, y_adult_test, 'sex')
runAdultWithSplitting(X_adult_expand, y_adult, X_adult_expand_test, y_adult_test, 'sex')
# run(X_adult_expand, y_adult, X_adult_expand_test, y_adult_test)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.8292875750536229
test: LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.8341965471447543
LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10) discrimination: 0.4612302495137398
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.8292875750536229
test: LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.8341965471447543
LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10) discrimination: 0.4612302495137398
--------------------------------------------------------------------------------
--------------------------------------------------------

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_

----------------------------------------
val: LR(solver=sag, fit_intercept=False, max_iter=1000, penalty=none, C=1e10): 0.8293870280620765
test: LR(solver=sag, fit_intercept=False, max_iter=1000, penalty=none, C=1e10): 0.8341965471447543
LR(solver=sag, fit_intercept=False, max_iter=1000, penalty=none, C=1e10) discrimination: 0.4615482145375872
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


KeyboardInterrupt: 

# German Dataset

In [ ]:
# load german dataset
df_german = load_german('datasets/german/german.data')
X_german = df_german.iloc[:, :-1]
y_german = df_german.iloc[:, -1]

X_german, X_german_test, y_german, y_german_test = train_test_split(X_german, y_german, test_size=0.3, random_state=0)

# expanding (one-hot encoding)

In [ ]:
X_german, X_german_test = encode_german(X_german, X_german_test)

X_german_all = X_german.append(X_german_test)
X_german_all_expand = pd.get_dummies(X_german_all)

# X_expand, X_expand_test = encode_adult(X_expand, X_expand_test)
X_german_expand = X_german_all_expand[0:X_german.shape[0]]
X_german_expand_test = X_german_all_expand[X_german.shape[0]:]

In [ ]:
X_german_expand = encode_german_all(X_german)
X_german_all_expand = pd.get_dummies(X_german_expand)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_german_encoded, y_german, test_size=0.3, shuffle=False)
# X_train, X_test, y_train, y_test = train_test_split(X_german, y_german, test_size=0.3, random_state=42)
X_german_all_expand

In [ ]:
# run(X_train, y_train, X_test, y_test, 'Age')
runGermanWithSplitting(X_german_all_expand, y_german, 12);
# run(X_german_expand, y_german, X_german_expand_test, y_german_test)

# The most similar model

In [ ]:
models = []
names = ['LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10)',
]

models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=10000, penalty='none'))

In [ ]:
# runAdultWithSplitting(X_expand, y_adult, X_expand_test, y_adult_test, 'sex')

In [ ]:
# runGermanWithSplitting(X_german_encoded, y_german, 'Age');

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_german_encoded, y_german, test_size=0.3, shuffle=False)
run(X_german_expand, y_german, X_german_expand_test, y_german_test, 12)